In [1]:
# Import dependencies and API key
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from config import gkey
gmaps.configure(gkey)

In [2]:
# Import previous data and create dataframe
weather_df = pd.read_csv("../weatherpy/output/weatherpy.csv", encoding="utf-8")
weather_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed
0,0,east london,IS,65.0000,-15.0000,1619287264,0,66,48.2,12.66
1,1,grand river south east,US,38.8755,-109.5012,1619287264,1,14,73.0,5.75
2,2,ushuaia,AR,-54.8000,-68.3000,1619286972,40,58,55.4,16.11
3,3,portland,US,45.5234,-122.6762,1619286600,90,81,50.0,9.22
4,4,roald,NO,62.5833,6.1333,1619287265,20,60,39.2,8.05


In [10]:
# Set humidity max as weight, use latitude and longitude as locations
humidity = weather_df["Humidity"].astype(float)
max_humidity = humidity.max()
location = weather_df[["Latitude", "Longitude"]]

# Configure gmaps
fig = gmaps.figure()

# Add heat layer to map
heat_layer = gmaps.heatmap_layer(location, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Narrow down cities to fit weather conditions
ideal_weather_df = weather_df.loc[(weather_df["Max Temperature"] > 70) & (weather_df["Max Temperature"] < 80) & (weather_df["Cloudiness"] == 0) & (weather_df["Wind Speed"] < 7) & (weather_df["Humidity"] < 80), :]

# Drop any rows will null values
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df.reset_index(inplace=True)
del ideal_weather_df["index"]
del ideal_weather_df["Unnamed: 0"]

# Display df
ideal_weather_df

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed
0,roebourne,AU,-20.7833,117.1333,1619287356,0,41,79.45,6.22
1,kohlu,PK,29.8962,69.2520,1619287403,0,28,72.27,3.53


In [5]:
# Set parameters, search Google Places, store hotels 
hotels = []

for i in range(len(ideal_weather_df)):
    lat = ideal_weather_df.loc[i]['Latitude']
    lng = ideal_weather_df.loc[i]['Longitude']

    paramaters = {"location": f"{lat},{lng}", "radius": 5000, "types" : "hotel", "key": gkey}
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(url, params=paramaters)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
        
# Add a "Hotel Name" column to the DataFrame
ideal_weather_df["Hotel Name"] = hotels
ideal_weather_df = ideal_weather_df.dropna(how='any')
ideal_weather_df

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temperature,Wind Speed,Hotel Name
0,roebourne,AU,-20.7833,117.1333,1619287356,0,41,79.45,6.22,Roebourne
1,kohlu,PK,29.8962,69.2520,1619287403,0,28,72.27,3.53,Kohlu


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Latitude", "Longitude"]]

In [11]:
# Plot markers on top of the heatmap.
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))